1. openai, streamlit, pytubefix 라이브러리 확인하기

In [ ]:
# 각 라이브러리가 설치되어 있는지 확인
!pip show openai streamlit pytubefix

2. 동영상 자동 자막 생성 웹 애플리케이션 만들기

In [ ]:
%%writefile ./project23/streamlit_app.py

import streamlit as st
from pytubefix import YouTube
import os
from openai import OpenAI
from pathlib import Path
import tempfile

# 1 메인 애플리케이션 main( ) 함수
def main():
    st.title("YouTube 영상 자막 생성기")
    st.write("YouTube 영상 주소를 입력하면 OpenAI Whisper API를 사용하여 SRT 자막을 생성합니다.")

    # 2 OpenAI API 초기화
    api_key = st.text_input("OpenAI API 키를 입력하세요", type="password")
    
    if api_key:
        client = OpenAI(api_key=api_key)
        
    youtube_url = st.text_input("YouTube 영상 URL을 입력하세요")
    
    if st.button("자막 생성") and youtube_url and api_key:
        try:
            # 3 YouTube 영상 다운로드 및 오디오 추출
            with st.spinner("음성 파일 다운로드 중..."):
                yt = YouTube(youtube_url)
                audio_stream = yt.streams.filter(only_audio=True).first()
                
                temp_dir = tempfile.mkdtemp()
                temp_audio_path = os.path.join(temp_dir, "audio.mp4")
                audio_stream.download(output_path=temp_dir, filename="audio.mp4")
                
            # 4 Whisper API를 사용한 자막 생성
            with st.spinner("자막 생성 중..."):
                with open(temp_audio_path, "rb") as audio_file:
                    transcript = client.audio.transcriptions.create(            
                        model="whisper-1",
                        file=audio_file,
                        response_format="srt"
                    )
                    
                # 5 결과 출력 및 다운로드 기능
                st.subheader("생성된 자막:")
                st.text_area("자막 미리보기", transcript, height=300)
                
                st.download_button(
                    label="자막 파일 다운로드",
                    data=transcript,
                    file_name="subtitle.srt",
                    mime="text/plain"
                )
                
            # 임시 파일 정리
            os.remove(temp_audio_path)
            os.rmdir(temp_dir)
            
        except Exception as e:
            st.error(f"오류가 발생했습니다: {str(e)}")
    
if __name__ == "__main__":
    main()

In [ ]:
# 스트림릿 앱 실행
!streamlit run ./project23/streamlit_app.py